# Download Price Data

Monthly CSVs in `data/<year>/PRICES_<year>-M<month>.csv`. Start date in `config/constants.py`. Set `FORCE_REDOWNLOAD = True` to re-download from start.

In [1]:
import sys
from pathlib import Path
from datetime import date

PROJECT_ROOT = Path.cwd().parent.parent if (Path.cwd().parent / "config").exists() else Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from research.config.constants import UNIVERSE, START_DATE as START_DATE_STR
from research.functions.data_source import fetch_prices
from research.functions.download_helper import (
    DownloadStats,
    create_new_file,
    determine_start_date,
    get_month_path,
    month_range,
    months_from,
    update_existing_file,
)

In [2]:
DATA_DIR = PROJECT_ROOT / "data"
START_DATE = date.fromisoformat(START_DATE_STR)
TICKERS = UNIVERSE
FORCE_REDOWNLOAD = False

DATA_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
today = date.today()

In [4]:
start_from, last_month = determine_start_date(DATA_DIR, START_DATE, FORCE_REDOWNLOAD)

# Short log lines
print(f"Fetching {start_from} → {today}", end="")
if last_month and not FORCE_REDOWNLOAD:
    print(f" (resume from {last_month.strftime('%Y-%m')})", end="")
print()

stats = DownloadStats()
for year, month in months_from(start_from, today):
    month_start, month_end = month_range(year, month)
    end_cap = min(month_end, today)
    if month_start > end_cap:
        continue
    path = get_month_path(DATA_DIR, year, month)
    label = f"{year}-M{month:02d}"
    if path.exists():
        updated, n = update_existing_file(path, end_cap, TICKERS, fetch_prices, stats)
        if updated:
            stats.updated += 1
            print(f"  {label}: +{n}")
    else:
        created, n = create_new_file(path, month_start, end_cap, TICKERS, fetch_prices, stats)
        if created:
            stats.updated += 1
            print(f"  {label}: {n} rows")

print(stats.log_summary())
print("Done.")

Fetching 2026-02-01 → 2026-02-10 (resume from 2026-02)
  2026-M02: +5
Updated: 1 | Up to date: 0 | No new data: 0
Done.
